## libraries and function 

In [1]:
!pip install impyute
!pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
import impyute as impy
from fancyimpute import IterativeSVD, SoftImpute, NuclearNormMinimization
import pandas as pd
import time 

     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 2.7 MB 39.8 MB/s 
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11353 sha256=ea9244c201c372eec0ca3ba26bfb53c2665a6417d1f2592636a1767b56eb682c
  Stored in directory: /root/.cache/pip/wheels/72/21/a8/a045cacd9838abd5643f6bfa852c0796a99d6b1494760494e0
Successfully built knnimpute
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


### MLE estimation function 

In [2]:
def diag_term(X,i):
  arr0 = X[:,i].flatten()
  arr = arr0[~np.isnan(arr0)]
  return np.var(arr)

def musMLE(X,y,G):
    n,p = X.shape[0], X.shape[1]
    f = lambda g: np.nanmean(X[y==g,:],axis=0)
    musMLE = np.array([f(g) for g in range(G)])    
    return musMLE.T

def Smle(X,y,musMLE,g):
    '''
    function to compute the covariance matrix for the g-th class
    X: input, should be a numpy array
    y: label
    G: number of classes
    g: class index
    output:
    - mus: each row is a class mean
    - S: common covariance matrix of class 1,2,..., G 
    '''
    epsilon = 1e-5 # define epsilon to put r down to 0 if r < epsilon
    Xg, yg = X[y==g,:], y[y==g]
    n,p = Xg.shape[0], Xg.shape[1] 
 
    S = np.diag([diag_term(Xg,i) for i in range(p)]) 

    for i in range(p):      
      for j in range(i):
        if ((S[i,i] == 0.) | (S[j,j] == 0.)):
          S[i,j] = S[j,i] = 0.
          continue

        mat = Xg[:,[i,j]]

        # drop rows with NA
        idx = ~np.isnan(mat).any(axis=1)
        mat, y_arr = mat[idx], yg[idx]
        A = mg = len(y_arr) 

        s11 = mg*np.var(mat[:,0])
        s22 = mg*np.var(mat[:,1])
        s12 = sum((mat[:,0]-musMLE[i,g])*(mat[:,1]-musMLE[j,g]))
        B = S[i,i]*S[j,j]*A - s22 * S[i,i] - s11 * S[j,j]
        coefficient = [-A, s12, B, s12*S[i,i]*S[j,j]]
        r = np.roots(coefficient)
        r = r[abs(np.imag(r)) < epsilon]
        r = np.real(r)
        r[abs(r) < epsilon] = 0
 
        if len(r)>1:
          condi_var = S[j,j] - r**2/S[i,i]
          eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var
          r = r[eta == max(eta[~np.isnan(eta)])]

        if len(r) > 1: 
          if sum(r==0.0) == len(r):
            r = 0.
          else:  
            w = np.cov(mat, rowvar=False)  
            #r = r[w[0,1]*r>=0]
            r = r[np.abs(r-w[0,1]).argmin()] # select r that is closet to w[0,1] 

        S[i,j] = S[j,i] = r
    return S

### compute_err function 

In [3]:
def err(mus, S, mus_est, S_est):
  er = [np.linalg.norm(mus_est-mus)/mus.size,
        np.linalg.norm(S_est.flatten().flatten()-S.flatten())/S.size]  
  return np.mean(er)  

def generate_nan(Xtrain, missing_rate):
  Xshape = Xtrain.shape
  na_id = np.random.randint(0,Xtrain.size,round(missing_rate*Xtrain.size))
  Xtr_nan = Xtrain.flatten()
  Xtr_nan[na_id] = np.nan 
  return Xtr_nan.reshape(Xshape) 

In [4]:
def compute_err_mle(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])
    # MLEs approach
    start = time.time()
    mus_mle = musMLE(Xtr_nan,y,G)
    S_mle = np.array([Smle(Xtr_nan,y,mus_mle, g) for g in range(G)])   
    mle_err = err(mus, S, mus_mle.T, S_mle)
    mle_time = time.time()-start  
    e_rate.append(mle_err)
  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate)

In [5]:
def compute_err_soft(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])

    start = time.time()
    Xtr_softimpute = SoftImpute(max_iters = 100).fit_transform(Xtr_nan)
    mus_softimpute = np.asarray([np.mean(Xtr_softimpute[ytrain==g,:], axis=0
                                         ) for g in np.arange(G)])
    S_softimpute = np.asarray([np.cov(Xtr_softimpute[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    print('mus soft impute', mus_softimpute)
    print('S soft impute', S_softimpute)
    softimpute_err =  err(mus, S, mus_softimpute, S_softimpute)
    softimpute_time = time.time()-start

    e_rate.append(softimpute_err)

  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate)

In [6]:
def compute_err_mice(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])

    start = time.time()
    Xtr_mice = IterativeImputer(max_iter=100).fit(Xtr_nan).transform(Xtr_nan)
    mus_mice = np.asarray([np.mean(Xtr_mice[ytrain==g,:], axis=0
                                   ) for g in np.arange(G)])
    S_mice = np.asarray([np.cov(Xtr_mice[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])

    mice_err = err(mus, S, mus_mice, S_mice)
    mice_time = time.time()-start

    e_rate.append(mice_err)

  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate)

# Heart

In [7]:
data = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECTF.train', header = None,sep=',')
print(data.head())
test = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECTF.test',
                     header=None, sep = ',')
data = pd.concat([data, test])
data = data.to_numpy()
X,y = data[:,1:], data[:,0]
X = X.astype(np.float32)
G = len(np.unique(y)) 
print(np.shape(X))
for g in range(G):
  print(sum(y==g))

   0   1   2   3   4   5   6   7   8   ...  36  37  38  39  40  41  42  43  44
0   1  59  52  70  67  73  66  72  61  ...  56  62  56  72  62  74  74  64  67
1   1  72  62  69  67  78  82  74  65  ...  71  63  60  69  73  67  71  56  58
2   1  71  62  70  64  67  64  79  65  ...  70  66  65  64  55  61  41  51  46
3   1  69  71  70  78  61  63  67  65  ...  61  66  65  72  73  68  68  59  63
4   1  70  66  61  66  61  58  69  69  ...  69  70  66  70  64  60  55  49  41

[5 rows x 45 columns]
(267, 44)
55
212


In [8]:
G = 2
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in log


array([[0.002, 0.   ],
       [0.003, 0.   ],
       [0.004, 0.   ],
       [0.005, 0.   ],
       [0.006, 0.   ]])

In [9]:
G = 2
mice_err = np.array([compute_err_mice(X, y, G, .2, runs = 10),
                    compute_err_mice(X, y, G, .35, runs = 10),
                    compute_err_mice(X, y, G, .5, runs = 10),
                    compute_err_mice(X, y, G, .65, runs = 10),
                    compute_err_mice(X, y, G, .8, runs = 10)])
mice_err.round(3)

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


array([[0.002, 0.   ],
       [0.003, 0.   ],
       [0.004, 0.   ],
       [0.004, 0.   ],
       [0.006, 0.   ]])

In [ ]:
G = 2
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])

In [12]:
soft_err.round(3)

array([[0.002, 0.   ],
       [0.003, 0.   ],
       [0.004, 0.   ],
       [0.005, 0.   ],
       [0.006, 0.   ]])

# Inosphere

In [ ]:
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data',
                  sep = ",", header = None)
# print(data.head())
data = pd.DataFrame.to_numpy(data)
X, y = data[:,:34].astype(np.float64), data[:,34]
le2 = LabelEncoder()
y = le2.fit_transform(y)
print(len(y))
X = np.delete(X,[0,1], axis = 1)
X.shape

351


(351, 32)

In [ ]:
G = 2
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

array([[0.003, 0.   ],
       [0.005, 0.001],
       [0.006, 0.   ],
       [0.008, 0.001],
       [0.008, 0.001]])

In [ ]:
G = 2
mice_err = np.array([compute_err_mice(X, y, G, .2, runs = 10),
                    compute_err_mice(X, y, G, .35, runs = 10),
                    compute_err_mice(X, y, G, .5, runs = 10),
                    compute_err_mice(X, y, G, .65, runs = 10),
                    compute_err_mice(X, y, G, .8, runs = 10)])
mice_err.round(3)

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


array([[0.004, 0.   ],
       [0.005, 0.   ],
       [0.006, 0.001],
       [0.007, 0.001],
       [0.009, 0.001]])

In [ ]:
G = 2
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])

Streaming output truncated to the last 5000 lines.
[SoftImpute] Iter 48: observed MAE=0.035688 rank=32
[SoftImpute] Iter 49: observed MAE=0.035679 rank=32
[SoftImpute] Iter 50: observed MAE=0.035671 rank=32
[SoftImpute] Iter 51: observed MAE=0.035663 rank=32
[SoftImpute] Iter 52: observed MAE=0.035656 rank=32
[SoftImpute] Iter 53: observed MAE=0.035649 rank=32
[SoftImpute] Iter 54: observed MAE=0.035642 rank=32
[SoftImpute] Iter 55: observed MAE=0.035636 rank=32
[SoftImpute] Iter 56: observed MAE=0.035630 rank=32
[SoftImpute] Iter 57: observed MAE=0.035624 rank=32
[SoftImpute] Iter 58: observed MAE=0.035618 rank=32
[SoftImpute] Iter 59: observed MAE=0.035613 rank=32
[SoftImpute] Iter 60: observed MAE=0.035608 rank=32
[SoftImpute] Iter 61: observed MAE=0.035604 rank=32
[SoftImpute] Iter 62: observed MAE=0.035599 rank=32
[SoftImpute] Iter 63: observed MAE=0.035595 rank=32
[SoftImpute] Iter 64: observed MAE=0.035591 rank=32
[SoftImpute] Iter 65: observed MAE=0.035587 rank=32
[SoftImpute] 

In [ ]:
soft_err.round(3)

array([[0.003, 0.   ],
       [0.005, 0.   ],
       [0.006, 0.   ],
       [0.008, 0.   ],
       [0.009, 0.   ]])

# seeds 

In [ ]:
data = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt',
                     sep = '\s+', header = None)
data = pd.DataFrame.to_numpy(data)
X,y = data[:,:7], data[:,7]-1 # reset the labels to go start from 0  

In [ ]:
G = 3
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

array([[0.005, 0.001],
       [0.007, 0.001],
       [0.008, 0.001],
       [0.01 , 0.001],
       [0.012, 0.001]])

In [ ]:
G = 3
mice_err = np.array([compute_err_mice(X, y, G, .2, runs = 10),
                    compute_err_mice(X, y, G, .35, runs = 10),
                    compute_err_mice(X, y, G, .5, runs = 10),
                    compute_err_mice(X, y, G, .65, runs = 10),
                    compute_err_mice(X, y, G, .8, runs = 10)])
mice_err.round(3)

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  -------
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  -------
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  -------
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  -------
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  -------
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  -------
/usr/local/lib/python3.7/dist-packages/s

array([[0.004, 0.001],
       [0.006, 0.001],
       [0.011, 0.002],
       [0.015, 0.003],
       [0.022, 0.005]])

In [ ]:
G = 3
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])
soft_err.round(3)

Streaming output truncated to the last 5000 lines.
   -0.02418619  0.15285866]
  [-0.03957577 -0.03810996 -0.04124457 -0.06249281 -0.02418619
    0.46838973 -0.02237923]
  [ 0.19843     0.2058273   0.00941881  0.26393482  0.15285866
   -0.02237923  0.26228359]]

 [[ 0.06495505  0.06075137  0.10723667  0.04046239  0.08525057
   -0.02386311  0.01783012]
  [ 0.06075137  0.06690432  0.04637308  0.05970779  0.06543228
   -0.01370614  0.03887698]
  [ 0.10723667  0.04637308  0.62935362 -0.07095395  0.24854886
   -0.10986922 -0.11366453]
  [ 0.04046239  0.05970779 -0.07095395  0.08743909  0.01504247
   -0.00439702  0.06960142]
  [ 0.08525057  0.06543228  0.24854886  0.01504247  0.14721301
   -0.03680235 -0.01145977]
  [-0.02386311 -0.01370614 -0.10986922 -0.00439702 -0.03680235
    0.6977335   0.05270192]
  [ 0.01783012  0.03887698 -0.11366453  0.06960142 -0.01145977
    0.05270192  0.089858  ]]]
[SoftImpute] Max Singular Value of X_init = 23.503812
[SoftImpute] Iter 1: observed MAE=0.028796 r

array([[0.007, 0.001],
       [0.01 , 0.001],
       [0.015, 0.002],
       [0.022, 0.002],
       [0.029, 0.002]])

# wine
The data set is also available in sklearn, as noted in the package's website. So, we load it directly from sklearn

In [ ]:
wine = datasets.load_wine()
X,y = wine.data, wine.target.ravel() 
# sum(y==0), sum(y==1), sum(y==2)

In [ ]:
G = 3
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

array([[0.005, 0.   ],
       [0.007, 0.   ],
       [0.008, 0.001],
       [0.011, 0.001],
       [0.013, 0.002]])

In [ ]:
G = 3
mice_err = np.array([compute_err_mice(X, y, G, .2, runs = 10),
                    compute_err_mice(X, y, G, .35, runs = 10),
                    compute_err_mice(X, y, G, .5, runs = 10),
                    compute_err_mice(X, y, G, .65, runs = 10),
                    compute_err_mice(X, y, G, .8, runs = 10)])
mice_err.round(3)

array([[0.006, 0.001],
       [0.009, 0.001],
       [0.012, 0.001],
       [0.014, 0.002],
       [0.021, 0.002]])

In [ ]:

G = 3
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])
soft_err.round(3)

Streaming output truncated to the last 5000 lines.
[SoftImpute] Iter 36: observed MAE=0.027020 rank=13
[SoftImpute] Iter 37: observed MAE=0.027023 rank=13
[SoftImpute] Iter 38: observed MAE=0.027026 rank=13
[SoftImpute] Iter 39: observed MAE=0.027029 rank=13
[SoftImpute] Iter 40: observed MAE=0.027031 rank=13
[SoftImpute] Iter 41: observed MAE=0.027033 rank=13
[SoftImpute] Iter 42: observed MAE=0.027035 rank=13
[SoftImpute] Iter 43: observed MAE=0.027037 rank=13
[SoftImpute] Iter 44: observed MAE=0.027038 rank=13
[SoftImpute] Iter 45: observed MAE=0.027039 rank=13
[SoftImpute] Iter 46: observed MAE=0.027041 rank=13
[SoftImpute] Iter 47: observed MAE=0.027042 rank=13
[SoftImpute] Iter 48: observed MAE=0.027043 rank=13
[SoftImpute] Iter 49: observed MAE=0.027045 rank=13
[SoftImpute] Iter 50: observed MAE=0.027046 rank=13
[SoftImpute] Iter 51: observed MAE=0.027048 rank=13
[SoftImpute] Iter 52: observed MAE=0.027049 rank=13
[SoftImpute] Iter 53: observed MAE=0.027051 rank=13
[SoftImpute] 

array([[0.007, 0.001],
       [0.012, 0.001],
       [0.016, 0.001],
       [0.02 , 0.001],
       [0.026, 0.002]])

# Iris
The data set is also available in sklearn, as noted in the package's website. So, we load it directly from sklearn

In [ ]:
iris = datasets.load_iris()
X,y = iris.data, iris.target.ravel() 
G = 3

In [ ]:
G = 3
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

array([[0.007, 0.002],
       [0.011, 0.002],
       [0.013, 0.002],
       [0.013, 0.002],
       [0.015, 0.002]])

In [ ]:
G = 3
mice_err = np.array([compute_err_mice(X, y, G, .2, runs = 10),
                    compute_err_mice(X, y, G, .35, runs = 10),
                    compute_err_mice(X, y, G, .5, runs = 10),
                    compute_err_mice(X, y, G, .65, runs = 10),
                    compute_err_mice(X, y, G, .8, runs = 10)])
mice_err.round(3)

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: Co

array([[0.008, 0.001],
       [0.013, 0.003],
       [0.021, 0.002],
       [0.034, 0.006],
       [0.046, 0.008]])

In [ ]:
G = 3
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])
soft_err.round(3)

Streaming output truncated to the last 5000 lines.
[SoftImpute] Iter 73: observed MAE=0.025657 rank=4
[SoftImpute] Iter 74: observed MAE=0.025657 rank=4
[SoftImpute] Iter 75: observed MAE=0.025657 rank=4
[SoftImpute] Iter 76: observed MAE=0.025657 rank=4
[SoftImpute] Iter 77: observed MAE=0.025657 rank=4
[SoftImpute] Iter 78: observed MAE=0.025657 rank=4
[SoftImpute] Iter 79: observed MAE=0.025657 rank=4
[SoftImpute] Iter 80: observed MAE=0.025658 rank=4
[SoftImpute] Iter 81: observed MAE=0.025658 rank=4
[SoftImpute] Iter 82: observed MAE=0.025658 rank=4
[SoftImpute] Stopped after iteration 82 for lambda=0.357876
mus soft impute [[-0.94628768  0.74289253 -1.22307152 -1.21841628]
 [ 0.16829999 -0.56673599  0.26620161  0.17569689]
 [ 0.85241371 -0.18699868  0.97585025  1.02836532]]
S soft impute [[[0.18367168 0.22959341 0.02142381 0.02519257]
  [0.22959341 0.69906106 0.0133427  0.03767691]
  [0.02142381 0.0133427  0.02033043 0.00905052]
  [0.02519257 0.03767691 0.00905052 0.02858089]]

 

array([[0.013, 0.003],
       [0.023, 0.004],
       [0.035, 0.005],
       [0.05 , 0.003],
       [0.063, 0.009]])